# Stock Prediction Model using Prophet and Yfinance

In [ ]:
import datetime
from pathlib import Path

import joblib
import pandas as pd
import yfinance as yf
from prophet import Prophet

from workspace.settings import ws_settings

In [ ]:
TODAY = datetime.date.today()
DEFAULT_TICKER = "GOOG"
DEFAULT_DAYS_TO_PREDICT = 21
MODELS_DIR = ws_settings.ws_root.joinpath("models")


def train(ticker=DEFAULT_TICKER):
    data = yf.download(ticker, "2020-01-01", TODAY.strftime("%Y-%m-%d"))
    data.head()
    data["Adj Close"].plot(title="Stock Adjusted Closing Price")

    df_forecast = data.copy()
    df_forecast.reset_index(inplace=True)
    df_forecast["ds"] = df_forecast["Date"]
    df_forecast["y"] = df_forecast["Adj Close"]
    df_forecast = df_forecast[["ds", "y"]]
    df_forecast

    model = Prophet()
    model.fit(df_forecast)
    joblib.dump(model, MODELS_DIR.joinpath(f"{ticker}_prediction.joblib"))


def predict(ticker=DEFAULT_TICKER, days=DEFAULT_DAYS_TO_PREDICT):
    model_file = MODELS_DIR.joinpath(f"{ticker}_prediction.joblib")
    if not model_file.exists():
        return None

    model = joblib.load(model_file)
    future = TODAY + datetime.timedelta(days=days)

    dates = pd.date_range(
        start="2020-01-01",
        end=future.strftime("%Y-%m-%d"),
    )
    df = pd.DataFrame({"ds": dates})

    forecast = model.predict(df)
    model.plot(forecast)
    # model.plot_components(forecast)

    return forecast.tail(days).to_dict("records")


def display_prediction_result(prediction_result):
    output = {}
    for data in prediction_result:
        date = data["ds"].strftime("%Y-%m-%d")
        output[date] = data["trend"]
    return output

In [ ]:
tickers_to_train = ["GOOG", "MSFT", "AAPL"]

for ticker in tickers_to_train:
    train(ticker)

In [ ]:
tickers_to_predict = ["GOOG", "MSFT", "AAPL"]

for ticker in tickers_to_train:
    prediction_result = predict(ticker, 60)

In [ ]:
# display_prediction_result(prediction_result)